In [1]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from utils.transformacion import *
from utils.modelos import *

VAR_SEED = 42
VAR_TESTSET_SIZE = 0.20
VAR_DIR_DATA_CLEAN = '../data/cleaning'

random.seed(VAR_SEED)
np.random.seed(VAR_SEED)

catalogo = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/catalogo.csv", sep=",", encoding="latin1")[['id_ejercicio', 'h1', 'h2', 'h3', 'h4', 's1', 's2', 's3', 's4', 'k1', 'k2', 'k3', 'k4']]
mf_dataset = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/mf_dataset.csv", sep=",", encoding="latin1")
dataset = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/dataset.csv", sep=",", encoding="latin1")
diagnostico = pd.read_csv(f"{VAR_DIR_DATA_CLEAN}/prueba_diagnostico.csv", sep=",", encoding="latin1")[['id_estudiante', 'score_a', 'score_p', 'score_d', 'score_s']]

2024-11-12 14:49:08.186475: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731433748.395115    2846 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731433748.445085    2846 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 14:49:08.874457: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Combinamos el conjunto de datos de usuarios y diagnósticos
df_users = pd.merge(mf_dataset, diagnostico, on='id_estudiante', how='left')
df_users.fillna(0, inplace=True)
df_users = df_users.astype(int)

# Preparamos el DataFrame de ítems
df_items = catalogo

# Definimos las dimensiones de entrada para los usuarios y los ejercicios
item_feature_size = len(df_items.columns) - 1   # Contamos todas las características de los ítems
user_feature_size = len(df_users.columns) - 1   # Contamos todas las características de los usuarios
embedding_size = 64                             # Tamaño del embedding

In [3]:
# Definimos la torre para los usuarios
class UserTower(nn.Module):
    def __init__(self, user_input_size, embedding_size):
        super(UserTower, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(user_input_size, 128),
            nn.ReLU(),
            nn.Linear(128, embedding_size)
        )
    
    def forward(self, x):
        return self.fc(x)

# Definimos la torre para los ejercicios
class ItemTower(nn.Module):
    def __init__(self, item_input_size, embedding_size):
        super(ItemTower, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(item_input_size, 128),
            nn.ReLU(),
            nn.Linear(128, embedding_size)
        )
    
    def forward(self, x):
        return self.fc(x)

# Definimos el modelo de dos torres
class TwoTowerModel(nn.Module):
    def __init__(self, user_input_size, item_input_size, embedding_size):
        super(TwoTowerModel, self).__init__()
        self.user_tower = UserTower(user_input_size, embedding_size)
        self.item_tower = ItemTower(item_input_size, embedding_size)
    
    def forward(self, user_input, item_input):
        # Obtener los embeddings para usuarios y ejercicios
        user_embedding = self.user_tower(user_input)
        item_embedding = self.item_tower(item_input)
        
        # Calcular la similitud como el producto punto entre los embeddings
        score = torch.sum(user_embedding * item_embedding, dim=1)
        
        return torch.sigmoid(score)


In [4]:
# Definir el modelo, optimizador y criterio (sin cambios)
model = TwoTowerModel(user_feature_size, item_feature_size, embedding_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

# Extraer las características y convertirlas a tensores (sin cambios)
item_input = torch.tensor(df_items.iloc[:, 1:].values).float()  # Datos de ejercicios
user_input = torch.tensor(df_users.iloc[:, 1:].values).float()  # Datos de usuarios

# Crear los pares de usuario e ítem (similar al Código 1)
num_users = len(df_users)
num_items = len(df_items)

# Expansión de las características para crear combinaciones
user_input_expanded = user_input.unsqueeze(1).expand(-1, num_items, -1).reshape(-1, user_feature_size)
item_input_expanded = item_input.repeat(num_users, 1).reshape(-1, item_feature_size)

# Asegúrate de que las dimensiones son correctas
print(f"User input expanded shape: {user_input_expanded.shape}")
print(f"Item input expanded shape: {item_input_expanded.shape}")

/home/edgar-ramxs/sistema-de-recomendacion-unab/.sistema-de-recomendacion-unab-env/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


User input expanded shape: torch.Size([40598, 57])
Item input expanded shape: torch.Size([40598, 12])


In [5]:
# Crear etiquetas (deberías tener información sobre las interacciones usuario-ejercicio)
labels = torch.randint(0, 2, (len(user_input_expanded),)).float()  # Aquí, el número de etiquetas debe ser el mismo que el número de pares

# Entrenar el modelo
epochs = 30
for epoch in range(epochs):
    optimizer.zero_grad()

    # Pasar las entradas por el modelo
    output = model(user_input_expanded, item_input_expanded)

    # Calcular la pérdida
    loss = criterion(output, labels)

    # Retropropagación
    loss.backward()

    # Actualizar los pesos
    optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 1/30, Loss: 0.6971
Epoch 2/30, Loss: 0.7287
Epoch 3/30, Loss: 0.6973
Epoch 4/30, Loss: 0.7006
Epoch 5/30, Loss: 0.7098
Epoch 6/30, Loss: 0.7045
Epoch 7/30, Loss: 0.6962
Epoch 8/30, Loss: 0.6936
Epoch 9/30, Loss: 0.6962
Epoch 10/30, Loss: 0.6991
Epoch 11/30, Loss: 0.6991
Epoch 12/30, Loss: 0.6967
Epoch 13/30, Loss: 0.6941
Epoch 14/30, Loss: 0.6930
Epoch 15/30, Loss: 0.6935
Epoch 16/30, Loss: 0.6947
Epoch 17/30, Loss: 0.6954
Epoch 18/30, Loss: 0.6952
Epoch 19/30, Loss: 0.6943
Epoch 20/30, Loss: 0.6933
Epoch 21/30, Loss: 0.6927
Epoch 22/30, Loss: 0.6927
Epoch 23/30, Loss: 0.6931
Epoch 24/30, Loss: 0.6934
Epoch 25/30, Loss: 0.6936
Epoch 26/30, Loss: 0.6933
Epoch 27/30, Loss: 0.6929
Epoch 28/30, Loss: 0.6926
Epoch 29/30, Loss: 0.6923
Epoch 30/30, Loss: 0.6923


In [6]:
def recommend_for_user(model, user_features, item_features, user_interacted_items, top_n=10):
    """
    Recomendaciones para un solo usuario basado en sus características, excluyendo ítems ya interactuados.
    
    Args:
        model: El modelo de recomendación preentrenado.
        user_features: Un array con las características del usuario.
        item_features: Un tensor con las características de todos los ítems.
        user_interacted_items: Lista de índices de los ítems que el usuario ya ha interactuado.
        top_n: Número de recomendaciones a devolver.

    Returns:
        recomendaciones: DataFrame con los ítems recomendados y sus puntuaciones.
    """
    # Asegurarse de que las características del usuario son un tensor y de la forma correcta
    user_input = torch.tensor(user_features).float().unsqueeze(0)  # Añadir una dimensión para el batch
    
    # Expandir las características del ítem para tener las combinaciones usuario-ítem
    num_items = item_features.size(0)
    item_input_expanded = item_features.repeat(user_input.size(0), 1)  # Repetir las características de los ítems para el usuario
    
    # Poner el modelo en modo de evaluación
    model.eval()
    
    # Obtener las puntuaciones de todos los ítems
    with torch.no_grad():
        item_scores = model(user_input, item_input_expanded)
    
    # Filtrar los ítems que ya fueron interactuados
    item_scores[user_interacted_items] = -float('inf')  # Asignar una puntuación baja a los ítems interactuados
    
    # Obtener los índices de los top_n ítems
    top_indices = torch.topk(item_scores, top_n).indices.squeeze().tolist()
    scores = item_scores[top_indices].tolist()
    
    # Crear el DataFrame con los resultados
    recommended_items = df_items.iloc[top_indices].copy()  # Asegurarse de que sea una copia y no una vista
    recommended_items.loc[:, 'Recommendation_Score'] = scores  # Añadimos las puntuaciones de recomendación
    
    return recommended_items




In [7]:
# Ejemplo de uso:

# Supongamos que 'new_user_features' es un array con las características del nuevo usuario
# 271 => 1,1,0,1,1,0,1,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,1,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0
# 271 => 5,5,2,3

exer = [1,1,0,1,1,0,1,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,1,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0]
prub = [5,5,2,3]

new_user_features = exer + prub  # Características del nuevo usuario
item_input_tensor = torch.tensor(df_items.iloc[:, 1:].values).float()  # Tensor de características de ítems

# Supongamos que el usuario ya ha interactuado con los ítems
user_interacted_items = [index for index, value in enumerate(exer) if value == 1]

# Obtener recomendaciones
recommended_df = recommend_for_user(model, new_user_features, item_input_tensor, user_interacted_items)
print("Recomendaciones del usuario:")
recommended_df

Recomendaciones del usuario:


,id_ejercicio,h1,h2,h3,h4,s1,s2,s3,s4,k1,k2,k3,k4,Recommendation_Score
34,34,0,0,1,1,0,0,0,0,1,1,0,0,0.550196
46,46,0,0,1,1,0,0,1,1,1,1,1,1,0.537555
43,43,0,0,1,1,0,0,1,1,1,1,1,1,0.537555
41,41,0,0,1,1,0,0,1,0,1,1,1,1,0.530971
45,45,0,0,1,1,0,0,1,0,1,1,1,1,0.530971
49,49,0,0,1,1,0,0,1,0,1,1,1,1,0.530971
15,15,0,0,1,1,0,1,0,1,0,1,1,1,0.530157
40,40,0,0,1,1,0,1,0,1,1,1,1,1,0.529826
27,27,0,0,1,1,0,0,1,0,0,1,1,1,0.526258
30,30,0,1,0,0,1,1,0,0,0,1,1,1,0.525496
